# Introduction: General information and Goals

Towards Data Science is one of most popular data science communities, articles published in their publication have influenced many data scientists all over the world. So I was interested in exploring trending topics, the most popular articles and their features. The general purpose of this notebook is to perform initial explanatory analysis of the data and give you some ideas for further research. 

My goals are:

- visualize data on volumes of articles published
- get insights on which articles were successful in the past years
- explore trends
- come up with further analysis ideas

In [ ]:
# Packages import

# Data manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Data visualization
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# Technical
import re
from PIL import Image

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Exploratory Data Analysis
## Data visualization

At first I load dataset from .csv file to DataFrame, because it is the most convenient format for data exploration. General information about dataset will help to dive deeper in data, understand data types and find null values.

In [ ]:
# Load dataset into DataFrame
data = pd.read_csv('../input/towards-data-science-articles-dataset-20102021/tds_data.csv')
data['publish_date'] = pd.to_datetime(data['publish_date'])
data.info()

In [ ]:
data1 = data[data['publish_date'].dt.year > 2016].copy()
pd.pivot_table(data1, index=data1['publish_date'].dt.year, values=['title'], columns='paid', 
               aggfunc='count').plot(kind = 'bar', figsize=(15,5))

plt.title("Published articles by Meduim Partnership Program participance", size=14)
plt.legend(['Free','Paid'])
plt.xlabel('Year')
plt.xticks(rotation='horizontal')
plt.ylabel('Number of articles')
plt.show()

I didn't plot data before year 2017, because it is less than 100 articles published in total during that time. 
As you can see number of paid articles significantly increased in 2019, not only in absolute values, but also relatively to number of free articles. In absolute numbers, number of free articles decreased since 2018.

For year 2021 you can see that: 
- number of published paid articles for only 20 days is nearly the same as for whole 2018 year
- looks like difference between paid and free articles will be the same as for 2019 and 2020

In [ ]:
pd.pivot_table(data1, index=data1['publish_date'].dt.year, values=['author'], 
               aggfunc='nunique').plot(kind = 'bar', figsize=(15,5), legend=False)

plt.title("Number of unique writers each year", size=14)
plt.xlabel('Year')
plt.xticks(rotation='horizontal')
plt.ylabel('Number of authors')
plt.show()

In [ ]:
authors = data.groupby(['author']).count().sort_values('title', ascending=False)['title']
print(f"General number of unique authors for all the time: {data['author'].nunique()}")
print(f"Author with highest number of articles - {authors.index[0]} ({authors[0]} articles in total)")
print(f"Author with second highest number of articles - {authors.index[1]} ({authors[1]} articles in total)")

In general, writers become more productive in 2020. As you can see number of articles in 2020 doubled, but number of authors didn't increased proportionally.

In [ ]:
pd.pivot_table(data1, index=data1['publish_date'].dt.year, values=['claps'], 
               aggfunc='sum').plot(kind = 'bar', figsize=(15,5), legend=False)

plt.title("Number of claps over time", size=14)
plt.xlabel('Year')
plt.xticks(rotation='horizontal')
plt.ylabel('Number of claps')
plt.show()

Number of claps slightly decreased in last two years. 
Having in mind that number of articles increased, readers might be less engaged now. 

Top 10 of most popular articles by number of claps (of all time)

In [ ]:
top10claps = data.sort_values('claps', ascending=False)[['publish_date','title', 'claps', 'paid']][:10]
top10claps.style.set_properties(subset=["title"], **{'text-align': 'left'}).hide_index()

Top 10 of most popular articles by number of responses (of all time)

In [ ]:
top10responses = data.sort_values('claps', ascending=False)[['publish_date','title', 'responses', 'paid']][:10]
top10responses.style.set_properties(subset=["title"], **{'text-align': 'left'}).hide_index()

The top 10 articles by both claps and responses are identical. Those are mostly articles published in 2018, so I think it will be interesting to see the most popular articles for year 2020.

Top 10 of most popular articles by number of claps (2020)

In [ ]:
top2020 = data[data['publish_date'].dt.year == 2020].sort_values('claps',
                                                                 ascending=False)[['title', 'claps', 'paid']][:10]
top2020.style.set_properties(subset=["title"], **{'text-align': 'left'}).hide_index()

## Insights


Let's take 100 most popular articles for each year starting from 2017 and try to get insights on their features.

In [ ]:
popular = data1.sort_values('claps', ascending=False).groupby(data1['publish_date'].dt.year, sort=False).head(100)
popular = popular.groupby(popular['publish_date'].dt.year).mean()
popular

In 2017 only 11% of top100 articles were paid, but through 2020 this number increased up to 90%. 
Average reading time got slight fluctuations, but, summing up, successful articles usually take 8-10 minutes of reading time.

Most popular words in titles (number of usage over years)

In [ ]:
# Get all words from the article titles and delete punctuation marks
all_words = [re.sub(r'[^\w\s]','',word) for title in data['title'].str.lower().str.split() for word in title]

# Count how many times word was used
count_dict = {}
for word in all_words:  
    if word in count_dict:  
        count_dict[word] += 1  
    else:  
        count_dict[word] = 1 

# Delete words that are not interesting (prepositions, etc.)
for word in ['to', 'on', 'in', 'at', 'of', 'for', 'with', 'a', 'an', 'the', 'and', 'by', 'it', 'are', 'as',
             'me', 'i', 'my', 'you', 'your', 'we', 'about', '']:
    count_dict.pop(word, None)

# Visualization using WordCloud
words_cloud = WordCloud(background_color="white",width=2000,height=1000, max_words=300,relative_scaling=0.5,
               normalize_plurals=False).generate_from_frequencies(count_dict)
plt.figure(figsize=(23,8), frameon=False)
plt.axis('off')
plt.imshow(words_cloud)
plt.show()

## Explore trends

Let's compare number of published articles with Google trends "towards data science" searches since 2017. It shows pretty the same trend as for number of published articles, but it has decreased recently. 

In [ ]:
df = pd.DataFrame()
df['dates'] = pd.Series(["2017-01-01", "2017-01-08", "2017-01-15", "2017-01-22", "2017-01-29", "2017-02-05", "2017-02-12", "2017-02-19", "2017-02-26", "2017-03-05", "2017-03-12", "2017-03-19", "2017-03-26", "2017-04-02", "2017-04-09", "2017-04-16", "2017-04-23", "2017-04-30", "2017-05-07", "2017-05-14", "2017-05-21", "2017-05-28", "2017-06-04", "2017-06-11", "2017-06-18", "2017-06-25", "2017-07-02", "2017-07-09", "2017-07-16", "2017-07-23", "2017-07-30", "2017-08-06", "2017-08-13", "2017-08-20", "2017-08-27", "2017-09-03", "2017-09-10", "2017-09-17", "2017-09-24", "2017-10-01", "2017-10-08", "2017-10-15", "2017-10-22", "2017-10-29", "2017-11-05", "2017-11-12", "2017-11-19", "2017-11-26", "2017-12-03", "2017-12-10", "2017-12-17", "2017-12-24", "2017-12-31", "2018-01-07", "2018-01-14", "2018-01-21", "2018-01-28", "2018-02-04", "2018-02-11", "2018-02-18", "2018-02-25", "2018-03-04", "2018-03-11", "2018-03-18", "2018-03-25", "2018-04-01", "2018-04-08", "2018-04-15", "2018-04-22", "2018-04-29", "2018-05-06", "2018-05-13", "2018-05-20", "2018-05-27", "2018-06-03", "2018-06-10", "2018-06-17", "2018-06-24", "2018-07-01", "2018-07-08", "2018-07-15", "2018-07-22", "2018-07-29", "2018-08-05", "2018-08-12", "2018-08-19", "2018-08-26", "2018-09-02", "2018-09-09", "2018-09-16", "2018-09-23", "2018-09-30", "2018-10-07", "2018-10-14", "2018-10-21", "2018-10-28", "2018-11-04", "2018-11-11", "2018-11-18", "2018-11-25", "2018-12-02", "2018-12-09", "2018-12-16", "2018-12-23", "2018-12-30", "2019-01-06", "2019-01-13", "2019-01-20", "2019-01-27", "2019-02-03", "2019-02-10", "2019-02-17", "2019-02-24", "2019-03-03", "2019-03-10", "2019-03-17", "2019-03-24", "2019-03-31", "2019-04-07", "2019-04-14", "2019-04-21", "2019-04-28", "2019-05-05", "2019-05-12", "2019-05-19", "2019-05-26", "2019-06-02", "2019-06-09", "2019-06-16", "2019-06-23", "2019-06-30", "2019-07-07", "2019-07-14", "2019-07-21", "2019-07-28", "2019-08-04", "2019-08-11", "2019-08-18", "2019-08-25", "2019-09-01", "2019-09-08", "2019-09-15", "2019-09-22", "2019-09-29", "2019-10-06", "2019-10-13", "2019-10-20", "2019-10-27", "2019-11-03", "2019-11-10", "2019-11-17", "2019-11-24", "2019-12-01", "2019-12-08", "2019-12-15", "2019-12-22", "2019-12-29", "2020-01-05", "2020-01-12", "2020-01-19", "2020-01-26", "2020-02-02", "2020-02-09", "2020-02-16", "2020-02-23", "2020-03-01", "2020-03-08", "2020-03-15", "2020-03-22", "2020-03-29", "2020-04-05", "2020-04-12", "2020-04-19", "2020-04-26", "2020-05-03", "2020-05-10", "2020-05-17", "2020-05-24", "2020-05-31", "2020-06-07", "2020-06-14", "2020-06-21", "2020-06-28", "2020-07-05", "2020-07-12", "2020-07-19", "2020-07-26", "2020-08-02", "2020-08-09", "2020-08-16", "2020-08-23", "2020-08-30", "2020-09-06", "2020-09-13", "2020-09-20", "2020-09-27", "2020-10-04", "2020-10-11", "2020-10-18", "2020-10-25", "2020-11-01", "2020-11-08", "2020-11-15", "2020-11-22", "2020-11-29", "2020-12-06", "2020-12-13", "2020-12-20", "2020-12-27"])
df['values']= pd.Series([0, 0, 0, 6, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 7, 0, 7, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 6, 0, 6, 6, 0, 0, 0, 6, 6, 6, 0, 0, 6, 7, 0, 6, 12, 6, 6, 0, 6, 6, 0, 0, 0, 12, 0, 6, 12, 6, 0, 18, 12, 24, 18, 12, 18, 6, 17, 6, 12, 12, 0, 24, 17, 11, 18, 12, 12, 6, 17, 11, 6, 6, 11, 29, 23, 23, 39, 17, 17, 22, 23, 17, 36, 26, 18, 23, 17, 45, 39, 34, 22, 39, 22, 17, 34, 40, 67, 57, 40, 30, 81, 29, 22, 28, 39, 28, 50, 44, 56, 68, 62, 67, 51, 29, 47, 64, 62, 17, 56, 59, 32, 70, 53, 38, 64, 42, 79, 58, 74, 52, 73, 21, 68, 84, 64, 59, 50, 83, 79, 92, 66, 80, 70, 70, 78, 81, 67, 47, 64, 64, 75, 68, 86, 36, 72, 87, 56, 61, 72, 60, 74, 58, 34, 34, 80, 55, 54, 88, 78, 74, 43, 100, 81, 62, 42, 77, 78, 51, 67, 36, 33, 30, 55, 79, 44, 35, 60, 50, 43])
df['dates'] = pd.to_datetime(df['dates'])

plt.figure(figsize=(15,5))
plt.plot(df['dates'], df['values'])
plt.show()

# Further analysis ideas
1. Try to predict number of articles that will be published in TDS in 2021
2. Predict trending topics
3. Extracting the main topics from dataset using LDA or other technology